<a href="https://colab.research.google.com/github/TurkuNLP/Turku-neural-parser-pipeline/blob/master/docs/tnpp_diaparse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turku Neural Parser Pipeline

* A mini-tutorial of the latest version of the pipeline

# INSTALL

* git clone the code
* cd to the directory
* and install all requirements

In [1]:
!git clone https://github.com/TurkuNLP/Turku-neural-parser-pipeline.git
%cd Turku-neural-parser-pipeline

Cloning into 'Turku-neural-parser-pipeline'...
remote: Enumerating objects: 1136, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 1136 (delta 95), reused 95 (delta 43), pack-reused 959
Receiving objects: 100% (1136/1136), 327.77 KiB | 12.14 MiB/s, done.
Resolving deltas: 100% (653/653), done.
/content/Turku-neural-parser-pipeline


In [2]:
!python3 -m pip install --upgrade pip
!python3 -m pip install --upgrade setuptools

     |████████████████████████████████| 1.6 MB 7.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
!python3 -m pip install -r requirements.txt

# FETCH MODEL

* At present, only the Finnish model is available for the diaparser-based version of the pipeline
* ...more to come...

In [4]:
!python3 fetch_models.py fi_tdt_dia

* The above command created the directory `models_fi_tdt_dia` with the model
* The file `models_fi_tdt_dia/pipelines.yaml` defines all the possible pipelines for the parser in this model
* Currently, only the `parse_plaintext` one works

# PARSE IN PYTHON

* You need to load and start the pipeline of choice
* Like so:

In [5]:
from tnparser.pipeline import read_pipelines, Pipeline

text1="Minulla on söpö koira. Se haukkuu, syö makkaraa,\
  jahtaa oravia ja tsillailee kanssani!"


# What pipelines do we have for the Finnish model?
available_pipelines=read_pipelines("models_fi_tdt_dia/pipelines.yaml")               # {pipeline_name -> its steps}
# This is a dictionary, its keys are the pipelines
print(list(available_pipelines.keys()))
# Instantiate one of the pipelines
p=Pipeline(available_pipelines["parse_plaintext"])    

['parse_plaintext', 'tag_plaintext', 'parse_sentlines', 'parse_wslines', 'parse_conllu', 'tokenize', 'parse_noisytext']


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/decorators.py:66: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `predict`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."
INFO:root:Loading model from /content/Turku-neural-parser-pipeline/models_fi_tdt_dia/Tagger/best.ckpt
INFO:filelock:Lock 140164513051728 acquired on /root/.cache/huggingface/transformers/e27939251243299384d3c49756d6710f25a683fa4d5e00e6f42fe6cc59202f07.1b2c5b5f39fed7ac39db55c0d2566730a96257ac7215ad6c2a8a109e2ccf1ccd.lock


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

In [6]:
print(p.parse(text1))

# newdoc
# newpar
# sent_id = 1
# text = Minulla on söpö koira.
1	Minulla	minä	PRON	_	Case=Ade|Number=Sing|Person=1|PronType=Prs	0	root	_	_
2	on	olla	AUX	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	1	cop:own	_	_
3	söpö	söpö	ADJ	_	Case=Nom|Degree=Pos|Number=Sing	4	amod	_	_
4	koira	koira	NOUN	_	Case=Nom|Number=Sing	1	nsubj:cop	_	SpaceAfter=No
5	.	.	PUNCT	_	_	1	punct	_	_

# sent_id = 2
# text = Se haukkuu, syö makkaraa, jahtaa oravia ja tsillailee kanssani!
1	Se	se	PRON	_	Case=Nom|Number=Sing|PronType=Dem	2	nsubj	_	_
2	haukkuu	haukkua	VERB	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	0	root	_	SpaceAfter=No
3	,	,	PUNCT	_	_	4	punct	_	_
4	syö	syödä	VERB	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	2	conj	_	_
5	makkaraa	makkara	NOUN	_	Case=Par|Number=Sing	4	obj	_	SpaceAfter=No
6	,	,	PUNCT	_	_	7	punct	_	SpacesAfter=\s\s
7	jahtaa	jahtaa	VERB	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	2	conj	_	_
8	oravia	orava